<a href="https://colab.research.google.com/github/arunm917/Climate-Action-Tool/blob/main/solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading packages and files

In [80]:
import numpy as np
import pandas as pd
import gdown
import regex as re
from scipy.optimize import minimize

In [81]:
# downloading file from gdrive
output = 'equations'
file_id = '1raendJQeKuzTduzPYIgcIXZYlRA7esAU' # cement
# file_id = '1shDdoDOjaa6lIn8zqRo38nmMm71de7mb' # steel
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1raendJQeKuzTduzPYIgcIXZYlRA7esAU
To: /content/equations
100%|██████████| 809/809 [00:00<00:00, 2.39MB/s]


DONE.


In [82]:
# downloading file from gdrive
output = 'parameters'
file_id = '1qC5eDYSwAqwYZ0QqlVpayiuQnix9e85K' # cement
# file_id = '1siz79kixFg13GPMp95d4HeUkPO27va1G' # steel
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qC5eDYSwAqwYZ0QqlVpayiuQnix9e85K
To: /content/parameters
100%|██████████| 433/433 [00:00<00:00, 1.32MB/s]


DONE.


In [83]:
# downloading file from gdrive
output = 'variables'
file_id = '1qAyqEmDgX0hvR2LN3moC6i3SDNrlylYZ' # cement
# file_id = '1sm04sQ_TnfesCuVrNUFz52SQHCGxZzgp' # steel
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qAyqEmDgX0hvR2LN3moC6i3SDNrlylYZ
To: /content/variables
100%|██████████| 347/347 [00:00<00:00, 376kB/s]


DONE.


# Processing files

In [84]:
parameters_df = pd.read_csv('parameters')

In [85]:
parameters_df

,parameters,values,type
0,phi_crusher,3.50,fixed
1,phi_rm,26.00,fixed
2,phi_cm,76.30,fixed
3,phi_phk,37.70,fixed
4,phi_cooler,37.70,fixed
5,phi_cementmill,30.00,fixed
6,w_add_ls,0.11,fixed
7,w_c_ls,0.06,fixed
8,w_c_coal,0.76,fixed
9,w_cl_rm,0.64,fixed


In [86]:
correct_params_df = parameters_df.loc[parameters_df['type'] == 'correct', ['parameters','values']]

In [87]:
correct_params_df

,parameters,values
15,eta_plant,0.6


In [88]:
variables_df = pd.read_csv('variables')
variables_df

,variables,values,type
0,m_ls,100,fixed
1,m_rm_gcv,0,float
2,m_rawmeal,0,float
3,m_pcoal,0,float
4,m_phk_air,0,float
5,m_hotclinker,0,float
6,m_stackgas,0,float
7,m_clinker,0,float
8,m_cement,0,float
9,m_cooler_air,0,float


In [89]:
correct_vars_df = variables_df.loc[variables_df['type'] == 'correct', ['variables','values']]

In [90]:
correct_vars_df

,variables,values


In [91]:
'''concatinating the dataframes containing correct variables and parameters'''

correct_params_df_copy = correct_params_df.copy()
correct_params_df_copy = correct_params_df_copy.rename(columns={'parameters': 'variables'})
correct_df = pd.concat([correct_vars_df, correct_params_df_copy])
correct_df = correct_df.reset_index(drop=True)

In [92]:
correct_df

,variables,values
0,eta_plant,0.6


In [93]:
decision_var_list = []
static_var_list = []

In [94]:
''' The following lines of code checks if the variable is set to fixed, float or correctable.
    fixed = variable value is fixed
    float = variable is part of the decision variable. Value assigned for variable is discarded
    correct = variable is changed by a small margin by making it a decision variable and applying a tolerance (using bounds).
    variable value is not discarded.'''

float_vars = variables_df.loc[variables_df['type'].isin(['float']), 'variables'].tolist()
decision_var_list.extend(float_vars) # float params is added to decision variables list
correct_vars = variables_df.loc[variables_df['type'].isin(['correct']), 'variables'].tolist()
decision_var_list.extend(correct_vars) # correct params are added to decision variables list
variables_df = variables_df[~variables_df['variables'].isin(float_vars + correct_vars)] # deleting parameters that are float and correct from parameters list. This is done so that they don't get substituted

In [95]:
print(decision_var_list)
print(correct_vars)
print(variables_df)

['m_rm_gcv', 'm_rawmeal', 'm_pcoal', 'm_phk_air', 'm_hotclinker', 'm_stackgas', 'm_clinker', 'm_cement', 'm_cooler_air', 'e_crusher', 'e_rawmill', 'e_coalmill', 'e_phk_total', 'e_cooler', 'e_cementmill', 'm_co2']
[]
  variables  values   type
0      m_ls     100  fixed


In [96]:
# static_var_list = list(static_var_df['variables'])
# decision_var_list = list(decision_var_df['variables'])

In [97]:
''' The following lines of code checks if the parameter is set to fixed, float or correctable.
    fixed = parameter value is fixed
    float = parameter is part of the decision variable. Value of parameter is discarded
    correct = parameter is changed by a small margin by making it a decision variable and applying a tolerance (using bounds).
    Parameter value is not discarded.'''

float_params = parameters_df.loc[parameters_df['type'].isin(['float']), 'parameters'].tolist()
decision_var_list.extend(float_params) # float params is added to decision variables list
correct_params = parameters_df.loc[parameters_df['type'].isin(['correct']), 'parameters'].tolist()
decision_var_list.extend(correct_params) # correct params are added to decision variables list
parameters_df = parameters_df[~parameters_df['parameters'].isin(float_params + correct_params)] # deleting parameters that are float and correct from parameters list. This is done so that they don't get substituted

In [98]:
print(correct_params)

['eta_plant']


In [99]:
parameters_df

,parameters,values,type
0,phi_crusher,3.50,fixed
1,phi_rm,26.00,fixed
2,phi_cm,76.30,fixed
3,phi_phk,37.70,fixed
4,phi_cooler,37.70,fixed
5,phi_cementmill,30.00,fixed
6,w_add_ls,0.11,fixed
7,w_c_ls,0.06,fixed
8,w_c_coal,0.76,fixed
9,w_cl_rm,0.64,fixed


In [100]:
print(decision_var_list)

['m_rm_gcv', 'm_rawmeal', 'm_pcoal', 'm_phk_air', 'm_hotclinker', 'm_stackgas', 'm_clinker', 'm_cement', 'm_cooler_air', 'e_crusher', 'e_rawmill', 'e_coalmill', 'e_phk_total', 'e_cooler', 'e_cementmill', 'm_co2', 'eta_kiln', 'eta_plant']


In [101]:
with open('equations', 'r') as f:
    # Read the equations line by line
    eq_lines = f.readlines()

In [102]:
print(eq_lines)

['eq1: (1 + w_add_ls) * m_ls - m_rm_gcv\n', 'eq2: m_rm_gcv - m_rawmeal\n', 'eq3: (beta_phk * w_cl_ls / gcv) * (m_ls / eta_plant) - m_pcoal  \n', 'eq4: m_cooler_air - w_c_coal * 23.0158 * m_pcoal \n', 'eq5: m_cooler_air - m_phk_air\n', 'eq6: (1 - w_cl_rm) * m_rawmeal + w_c_coal * m_pcoal + m_phk_air - m_stackgas\n', 'eq7: w_cl_rm * m_rawmeal + (1 - w_c_coal) * m_pcoal - m_clinker\n', 'eq8: m_hotclinker - m_clinker\n', 'eq9: m_clinker + w_gy_ls * m_ls + w_fly_ls * m_ls - m_cement\n', 'eq10: (m_ls * mol_wt_co2 / 100) + (w_c_coal * eta_kiln * mol_wt_co2 / mol_wt_c) * m_pcoal - m_co2 \n', 'eq11: e_crusher - phi_crusher * m_rm_gcv\n', 'eq12: e_rawmill - phi_rm * m_rawmeal\n', 'eq13: e_coalmill - phi_cm * m_pcoal\n', 'eq14: e_phk_total - beta_phk * 1000 * m_clinker \n', 'eq15: e_cooler - phi_cooler * m_hotclinker\n', 'eq16: e_cementmill - phi_cementmill * m_cement\n']


In [103]:
# Create a list to store the equations
eq_list = []

# Loop through the equation lines
for eq_line in eq_lines:
    # Split the line into the equation name and the equation expression
    eq_name, eq_expr = eq_line.strip().split(':')
    # Convert the tuple of symbols to a single expression
    eq_list.append(eq_expr)
    # Add the equation to the dictionary
print(eq_list)

[' (1 + w_add_ls) * m_ls - m_rm_gcv', ' m_rm_gcv - m_rawmeal', ' (beta_phk * w_cl_ls / gcv) * (m_ls / eta_plant) - m_pcoal', ' m_cooler_air - w_c_coal * 23.0158 * m_pcoal', ' m_cooler_air - m_phk_air', ' (1 - w_cl_rm) * m_rawmeal + w_c_coal * m_pcoal + m_phk_air - m_stackgas', ' w_cl_rm * m_rawmeal + (1 - w_c_coal) * m_pcoal - m_clinker', ' m_hotclinker - m_clinker', ' m_clinker + w_gy_ls * m_ls + w_fly_ls * m_ls - m_cement', ' (m_ls * mol_wt_co2 / 100) + (w_c_coal * eta_kiln * mol_wt_co2 / mol_wt_c) * m_pcoal - m_co2', ' e_crusher - phi_crusher * m_rm_gcv', ' e_rawmill - phi_rm * m_rawmeal', ' e_coalmill - phi_cm * m_pcoal', ' e_phk_total - beta_phk * 1000 * m_clinker', ' e_cooler - phi_cooler * m_hotclinker', ' e_cementmill - phi_cementmill * m_cement']


In [104]:
# Creating dictionary for parameters and their values
param_fixed_dict = dict(zip(parameters_df['parameters'],parameters_df['values']))
print(param_fixed_dict)

{'phi_crusher': 3.5, 'phi_rm': 26.0, 'phi_cm': 76.3, 'phi_phk': 37.7, 'phi_cooler': 37.7, 'phi_cementmill': 30.0, 'w_add_ls': 0.11, 'w_c_ls': 0.06, 'w_c_coal': 0.76, 'w_cl_rm': 0.64, 'w_air_ls': 1.64, 'w_gy_ls': 0.05, 'w_fly_ls': 0.33, 'w_cl_ls': 0.71, 'beta_phk': 2.97, 'gcv': 25.0, 'mol_wt_co2': 44.0, 'mol_wt_c': 12.0}


In [105]:
# Creating dictionary for variables and their values
var_fixed_dict = dict(zip(variables_df['variables'],variables_df['values']))
print(var_fixed_dict)

{'m_ls': 100}


In [106]:
correct_dict = dict(zip(correct_df['variables'],correct_df['values']))
print(correct_dict)

{'eta_plant': 0.6}


In [108]:
# Substituting parameters in equation with their values
modified_eq_list_param = []
for eq in eq_list:
  for key in param_fixed_dict:
    pattern = r'\b' + re.escape(key) + r'\b'
    if re.search(pattern, eq):
        value = param_fixed_dict.get(key)
        # eq = eq.replace(key, str(param_fixed_dict.get(key)))
        eq = re.sub(pattern, str(value), eq)
  modified_eq_list_param.append(eq)
print(modified_eq_list_param)

[' (1 + 0.11) * m_ls - m_rm_gcv', ' m_rm_gcv - m_rawmeal', ' (2.97 * 0.71 / 25.0) * (m_ls / eta_plant) - m_pcoal', ' m_cooler_air - 0.76 * 23.0158 * m_pcoal', ' m_cooler_air - m_phk_air', ' (1 - 0.64) * m_rawmeal + 0.76 * m_pcoal + m_phk_air - m_stackgas', ' 0.64 * m_rawmeal + (1 - 0.76) * m_pcoal - m_clinker', ' m_hotclinker - m_clinker', ' m_clinker + 0.05 * m_ls + 0.33 * m_ls - m_cement', ' (m_ls * 44.0 / 100) + (0.76 * eta_kiln * 44.0 / 12.0) * m_pcoal - m_co2', ' e_crusher - 3.5 * m_rm_gcv', ' e_rawmill - 26.0 * m_rawmeal', ' e_coalmill - 76.3 * m_pcoal', ' e_phk_total - 2.97 * 1000 * m_clinker', ' e_cooler - 37.7 * m_hotclinker', ' e_cementmill - 30.0 * m_cement']


In [109]:
# Substituting variables in equation with their values
modified_eq_list_var = [] # list after substituting for variables with corresponding values
for eq in modified_eq_list_param:
  for key in var_fixed_dict:
    if key in eq:
        value = var_fixed_dict.get(key)
        eq = eq.replace(key, str(var_fixed_dict.get(key)))
  modified_eq_list_var.append(eq)
print(modified_eq_list_var)

[' (1 + 0.11) * 100 - m_rm_gcv', ' m_rm_gcv - m_rawmeal', ' (2.97 * 0.71 / 25.0) * (100 / eta_plant) - m_pcoal', ' m_cooler_air - 0.76 * 23.0158 * m_pcoal', ' m_cooler_air - m_phk_air', ' (1 - 0.64) * m_rawmeal + 0.76 * m_pcoal + m_phk_air - m_stackgas', ' 0.64 * m_rawmeal + (1 - 0.76) * m_pcoal - m_clinker', ' m_hotclinker - m_clinker', ' m_clinker + 0.05 * 100 + 0.33 * 100 - m_cement', ' (100 * 44.0 / 100) + (0.76 * eta_kiln * 44.0 / 12.0) * m_pcoal - m_co2', ' e_crusher - 3.5 * m_rm_gcv', ' e_rawmill - 26.0 * m_rawmeal', ' e_coalmill - 76.3 * m_pcoal', ' e_phk_total - 2.97 * 1000 * m_clinker', ' e_cooler - 37.7 * m_hotclinker', ' e_cementmill - 30.0 * m_cement']


In [110]:
''' Updating the equations with the equations for data reconsiliation'''

for key, value in correct_dict.items():
  eq = f'{key} - {value}'
  modified_eq_list_var.append(eq)

In [111]:
for item in modified_eq_list_var:
  print(item)

 (1 + 0.11) * 100 - m_rm_gcv
 m_rm_gcv - m_rawmeal
 (2.97 * 0.71 / 25.0) * (100 / eta_plant) - m_pcoal
 m_cooler_air - 0.76 * 23.0158 * m_pcoal
 m_cooler_air - m_phk_air
 (1 - 0.64) * m_rawmeal + 0.76 * m_pcoal + m_phk_air - m_stackgas
 0.64 * m_rawmeal + (1 - 0.76) * m_pcoal - m_clinker
 m_hotclinker - m_clinker
 m_clinker + 0.05 * 100 + 0.33 * 100 - m_cement
 (100 * 44.0 / 100) + (0.76 * eta_kiln * 44.0 / 12.0) * m_pcoal - m_co2
 e_crusher - 3.5 * m_rm_gcv
 e_rawmill - 26.0 * m_rawmeal
 e_coalmill - 76.3 * m_pcoal
 e_phk_total - 2.97 * 1000 * m_clinker
 e_cooler - 37.7 * m_hotclinker
 e_cementmill - 30.0 * m_cement
eta_plant - 0.6


# Solving the optimization problem

In [112]:
# Define the objective function
def objective_function(decision_variables):
    # x, y, z, w = initial_guess
    for index, variable in enumerate(decision_var_list):
      globals()[variable] = decision_variables[index]

    equations = []
    for equation in modified_eq_list_var:
      equations.append(eval(equation))
    # print(equations)
    squared_errors = [result**2 for result in equations]
    return sum(squared_errors)

In [113]:
def constraints(decision_variables):

    for index, variable in enumerate(decision_var_list):
      globals()[variable] = decision_variables[index]

    constraints = []
    for equation in modified_eq_list_var:
      constraints.append(eval(equation))
    return constraints

In [114]:
# Initial guess for the decision variables
initial_guess = [1] * len(decision_var_list)

# Bounds for the decision variables
# bounds = [(0, 1000)]* len(variables_list)

# If type of decision variable is correctable the bounds are adjusted to a tolerance
tol = 0.2
bounds = []
for var in decision_var_list:
  if var in correct_params:
    value = correct_params_df.loc[correct_params_df['parameters'] == var, 'values'].values[0]
    # value = param_dict[var]
    lower_bound = value * (1 - tol)
    upper_bound = value * (1 + tol)
    bounds.append((lower_bound, upper_bound))

  elif var in correct_vars:
    value = correct_vars_df.loc[correct_vars_df['variables'] == var, 'values'].values[0]
    # value = param_dict[var]
    lower_bound = value * (1 - tol)
    upper_bound = value * (1 + tol)
    bounds.append((lower_bound, upper_bound))

  else:
    bounds.append((0, float('inf')))

result = minimize(objective_function, initial_guess, bounds = bounds, constraints={'type': 'eq', 'fun': constraints})

In [115]:
# Extract the optimal solution
optimal_solution = result.x
print("Optimal solution:", optimal_solution)

Optimal solution: [1.11000000e+02 1.11000000e+02 1.40580000e+01 2.45902648e+02
 7.44139200e+01 2.96546728e+02 7.44139200e+01 1.12413920e+02
 2.45902648e+02 3.88500000e+02 2.88600000e+03 1.07262540e+03
 2.21009342e+05 2.80540478e+03 3.37241760e+03 7.92834319e+01
 9.00662870e-01 6.00000000e-01]


# Print solution

In [116]:
for variable, value in zip(decision_var_list, optimal_solution):
    value = round(value,2)
    print(f"{variable}: {value}")

m_rm_gcv: 111.0
m_rawmeal: 111.0
m_pcoal: 14.06
m_phk_air: 245.9
m_hotclinker: 74.41
m_stackgas: 296.55
m_clinker: 74.41
m_cement: 112.41
m_cooler_air: 245.9
e_crusher: 388.5
e_rawmill: 2886.0
e_coalmill: 1072.63
e_phk_total: 221009.34
e_cooler: 2805.4
e_cementmill: 3372.42
m_co2: 79.28
eta_kiln: 0.9
eta_plant: 0.6
